In [20]:
from bs4 import BeautifulSoup
import lxml
import requests
import pandas as pd
import numpy as np

In [21]:
base_url = requests.get('https://www.indeed.com/cmp/Google/reviews?fcountry=ALL&start=', timeout=5)

print(base_url.text)

<!doctype html>
<html lang="en" dir="ltr">
<head>
    <title>Working at Google: 4,051 Reviews | Indeed.com</title>
    <meta http-equiv="content-type" content="text/html;charset=UTF-8">
        <meta name="description" content="4,051 reviews from Google employees about Google culture, salaries, benefits, work-life balance, management, job security, and more.">
        <meta name="viewport" content="width=device-width, initial-scale=1">

        <link rel="canonical" href="https://www.indeed.com/cmp/Google/reviews?fcountry&#61;ALL">
        <link rel="next" href="https://www.indeed.com/cmp/Google/reviews?fcountry&#61;ALL&amp;start&#61;20">
            <link rel="alternate" href="https://ar.indeed.com/cmp/Google/reviews?fcountry&#61;ALL" hreflang="es-AR">
            <link rel="alternate" href="https://au.indeed.com/cmp/Google/reviews?fcountry&#61;ALL" hreflang="en-AU">
            <link rel="alternate" href="https://be.indeed.com/cmp/Google/reviews?fcountry&#61;ALL" hreflang="nl-BE">
  

In [4]:
def parse(full_url):
    page_content = BeautifulSoup(full_url.content, 'lxml')
    containers = page_content.findAll('div',{'class':'cmp-Review-container'})
    df = pd.DataFrame(columns = ['rating', 'rating_title',  'rating_description','rating_pros', 'rating_cons'])
    
    for item in containers:
        try:
            rating = item.find('div', 
                     {'class': 'cmp-ReviewRating-text'}).text.replace('\n', '')
        except:
            rating = None
        try:
            rating_title = item.find('div', 
                           {'class': 'cmp-Review-title'}).text.replace('\n', '')
        except:
            rating_title = None
        try:
            rating_description = item.find('span', 
                                 {'itemprop': 'reviewBody'}).text.replace('\r', '. ')
        except:
            rating_description = None
        try:
            rating_pros = item.find('div', 
                          {'class': 'cmp-ReviewProsCons-prosText'}).text.replace('\n', '')
        except:
            rating_pros = None
        try:
            rating_cons = item.find('div', 
                          {'class': 'cmp-ReviewProsCons-consText'}).text.replace('\n', '')
        except:
            rating_cons = None
        df = df.append({'rating': rating, 
             'rating_title': rating_title, 
             'rating_description': rating_description,
             'rating_pros': rating_pros, 
             'rating_cons': rating_cons}, ignore_index=True)
    return df

In [18]:
base_url = 'https://www.indeed.com/cmp/Google/reviews?fcountry=ALL&start='
all_reviews_df = pd.DataFrame(columns = ['rating', 'rating_title', 'rating_description','rating_pros', 'rating_cons'])
num_reviews = 20
# you can adjust this number on how many reviews you which to scrape
while num_reviews < 100:  
    
    full_url = base_url + str(num_reviews)
    
    get_url = requests.get(full_url, timeout=5)  
        
    partial_reviews_df = parse(get_url)   
    # pd.concat([all_reviews_df,partial_reviews_df], ignore_index=True) 
    all_reviews_df = all_reviews_df.append(partial_reviews_df, ignore_index=True) 
    num_reviews += 20
    

C:\Users\gajavegr\AppData\Local\Temp/ipykernel_26204/2518542415.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_reviews_df = all_reviews_df.append(partial_reviews_df, ignore_index=True)
C:\Users\gajavegr\AppData\Local\Temp/ipykernel_26204/2518542415.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_reviews_df = all_reviews_df.append(partial_reviews_df, ignore_index=True)
C:\Users\gajavegr\AppData\Local\Temp/ipykernel_26204/2518542415.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_reviews_df = all_reviews_df.append(partial_reviews_df, ignore_index=True)
C:\Users\gajavegr\AppData\Local\Temp/ipykernel_26204/2518542415.py:13: FutureWarning: The frame.append method is deprecated and will be removed from

In [19]:
all_reviews_df.head(3)

,rating,rating_title,rating_description,rating_pros,rating_cons


In [9]:
all_reviews_df.to_csv('indeed_scrape.csv')